# Install and Load Packages

In [1]:
!pip install transformers
!pip install datasets
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
import torch.nn as nn
import tqdm
import pandas as pd
import faiss
import faiss.contrib.torch_utils

from transformers import DPRContextEncoder, DPRContextEncoderTokenizerFast
from datasets import Dataset
from google.colab import auth, drive
from google.cloud import bigquery

In [3]:
auth.authenticate_user()
print('Authenticated')

drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/nlp/data/'

Authenticated
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Data

In [4]:
project_id = 'calcium-vial-368801'
client = bigquery.Client(project=project_id)

In [5]:
wiki_passages = client.query('''
SELECT DISTINCT doc_id, title, text
FROM `calcium-vial-368801.prod_datasets.final_wikipedia_documents_3`
LIMIT 250000
''').to_dataframe()

In [6]:
wiki_passages['passage_append'] = wiki_passages['title'] + ' [SEP] ' + wiki_passages['text']

# Data Loader

In [7]:
ctx_tokenizer = DPRContextEncoderTokenizerFast.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


In [8]:
class MyDataset(Dataset):
    def __init__(self, dataframe, p_tokenizer):
        self.dataframe = dataframe
        self.p_tokenizer = p_tokenizer

        self.p_embed = p_tokenizer(
            self.dataframe['passage_append'].tolist(),
            return_tensors='pt',
            truncation=True,
            max_length=512,
            padding='max_length'
        )

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        return self.p_embed[index]


def collate_fn(batch):
    batchsize = len(batch)

    # Batch_size x 3 x 512 (max_passage_len)
    ctx_tensor = torch.LongTensor(
        [[sample.ids, sample.attention_mask, sample.type_ids] for sample in batch]
        )

    return ctx_tensor

BATCH_SIZE = 20

dataloader_train = torch.utils.data.DataLoader(
    MyDataset(wiki_passages, ctx_tokenizer),
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn
    )

In [9]:
len(dataloader_train)

12500

In [10]:
import gc

del ctx_tokenizer

gc.collect()
torch.cuda.empty_cache()

# Model

In [11]:
# https://discuss.huggingface.co/t/finetuning-dpr-on-custom-dataset/4170
ctx_model = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base').to("cuda")

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [12]:
gc.collect()
torch.cuda.empty_cache()

In [13]:
res = faiss.StandardGpuResources()
index = faiss.read_index(data_path + 'nq_train_qg_25_beam_passage_index')
gpu_index = faiss.index_cpu_to_gpu(res, 0, index)

In [14]:
class PassageEncoder(nn.Module):
    def __init__(self, p_encoder, index):
        super().__init__()
        self.p_encoder = p_encoder
        self.index = index

    def forward(self, passage):
        """
        args
            - passages: batchsize x 3 x passage_len matrix of passage tokens, token_ids, and attention_mask
        
        returns:
            - passage: batchsize x passage_embed_dim tensor of dense passage embeddings
        """

        self.index.add(self.p_encoder(passage[:, 0, :], passage[:, 1, :], passage[:, 2, :]).pooler_output.contiguous())


In [15]:
pEncoder = PassageEncoder(ctx_model, gpu_index)

# 5327 total batches
for i in tqdm.notebook.tqdm(dataloader_train, total=len(dataloader_train)):
    pEncoder(i.to("cuda"))
    torch.cuda.empty_cache()

  0%|          | 0/12500 [00:00<?, ?it/s]

In [16]:
faiss.write_index(faiss.index_gpu_to_cpu(gpu_index), data_path + 'wiki_nq_train_qg_25_beam_passage_index')
print(gpu_index.ntotal)

276634


In [17]:
# Make an index with the passages and queries
import json

with open(data_path + 'nq_train_qg_25_beam_passage_idx2docid.json') as f:
    old_index = json.loads(f.read())

In [18]:
print(len(old_index))
print(old_index['26633'])

26634
31917


In [19]:
import numpy as np

for i in np.arange(len(old_index), gpu_index.ntotal):
    old_index[str(i)] = wiki_passages.doc_id[i - len(old_index)]

In [20]:
with open(data_path + 'wiki_nq_train_qg_25_beam_passage_idx2docid.json', 'w') as f:
    f.write(json.dumps(old_index))